In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv("../input/jane-street-market-prediction/train.csv")

In [ ]:
data['resp_weight']=data['resp']*data['weight']

In [ ]:
data["action"] = (data["resp_weight"] > 0).astype('int')

In [ ]:
data['action'].value_counts()

In [ ]:
features = [col for col in data.columns if "feature" in col]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data[features], data["action"], test_size=0.4, random_state=42)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:

from lightgbm import LGBMClassifier
model = LGBMClassifier()
model.fit(X_train,y_train)
pred=model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, pred, labels=[True,False]))

In [ ]:
import janestreet
# Create submission
env = janestreet.make_env()
iter_test = env.iter_test()
for (test_df, sample_prediction_df) in iter_test: 
    X_test = test_df.loc[:, features]
    X_test = X_test.fillna(method='bfill')
    X_test = X_test.fillna(method='ffill')
    sample_prediction_df.action = model.predict(X_test)
    env.predict(sample_prediction_df)